In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
data = pd.read_csv("data/preprocessed_train.csv")

In [3]:
data.head()

,MasterDataTime,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,EDT,Hour,Weekday,AMorPM
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15 21:26:21,21,Monday,PM
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05 20:52:16,20,Tuesday,PM
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18 04:35:00,4,Thursday,AM
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21 08:30:42,8,Saturday,AM
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09 11:51:00,11,Tuesday,AM


In [4]:
features_of_interest = ["passenger_count", "Hour","Weekday","AMorPM","fare_amount"]
df = data[features_of_interest].copy()

In [5]:
df.head()

,passenger_count,Hour,Weekday,AMorPM,fare_amount
0,1,21,Monday,PM,4.5
1,1,20,Tuesday,PM,16.9
2,2,4,Thursday,AM,5.7
3,1,8,Saturday,AM,7.7
4,1,11,Tuesday,AM,5.3


In [6]:
for col in ["Weekday","AMorPM"]:
    df[col] = df[col].astype("category").cat.codes

df.head()

,passenger_count,Hour,Weekday,AMorPM,fare_amount
0,1,21,1,1,4.5
1,1,20,5,1,16.9
2,2,4,4,0,5.7
3,1,8,2,0,7.7
4,1,11,5,0,5.3


In [7]:
feature = df.drop("fare_amount", axis=1).copy()
target = df.fare_amount.copy()

## Setting up a Basic Linear Regression Model

### Get the Train Set

In [8]:
X_train, X_eval, y_train, y_eval = train_test_split(feature,
                                                   target,
                                                   test_size=0.1,
                                                   random_state=0,
                                                   )

### Get the Validation and the Test set

In [9]:
X_valid, X_test, y_valid, y_test= train_test_split(X_eval,
                                                  y_eval,
                                                  test_size=0.5,
                                                  random_state=0) 

### Modelling

In [10]:
baseline_LRO = LinearRegression(normalize=True)

In [11]:
model = baseline_LRO.fit(X_train, y_train)

#### The R2 score is awful.

In [12]:
model.score(X_train, y_train)*100

0.06462286984936227

#### Root Mean Squared Error vs The Mean of the Fare

In [13]:

pred = baseline_LRO.predict(X_valid)

np.sqrt(mean_squared_error(y_valid, pred))

9.66495969297465

In [14]:
data.fare_amount.mean()

11.348078720000004

### The above model can be a baseline only insofar as it predicts very close to the mean of the fare variable. To make this baseline more "usable", the lat and longitude